<a href="https://colab.research.google.com/github/jufabeck2202/KI-Lab/blob/main/%5BKI_Lab2021%5D_Assignment_4_OWCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview:

• In this assignment we want to build a model that can generate German nouns
given a prefix as input.
Example: Input „Jahr" → Output: „Jahrmarkt“ .

• For this purpose, we load German language data and filter it for nouns with spaCy.

• We then use a simple RNN to learn to generated names.

• In a second step, we switch to a more sophisticated LSTM model and compare the
results.


1. Go to the following Git repo which contains a dataset of German articles:
https://github.com/tblock/10kGNAD
2. Download the file train.csv	from there and import it into a notebook.
3. Drop the label and only use the sentence in each line (the part after the first ; ).
4. Use spaCy to get all nouns in the whole file.
5. Filter out the nouns that have characters wich are not in all_letters (see
code snippet below). The remaining set of nouns constitute our training data.
6. Go through the PyTorch tutorial on name generation. Clone the notebook, execute it
and understand what is going on.
7. Use the same architecture to train a language model on the nouns that were extracted
from the German dataset in step 1 to 5. (Note: We do not need a „category“, you have
to remove everything that belongs to „category“ from the code).
8. Extend the sampling logic in the sample function sucht that:
  
  i. The model extends any given prefix string by first feeding the prefix into the RNN
and then start generating new characters. (Currently the model only samples based
on a given starting letter).
  
  ii. Currently the model uses the highest score during sampling. Change that into
drawing a real sample from the softmax distribution.
9. Choose 10 random words from the training data and randomly cut-off a prefix. For
every of those prefixes give an output for:
  1. The model extending the prefix with the max score approach.
  2. The mode extending the prefix with a real sampling approach.
  3. Randomly pick characters from all_letters to extend the prefix (this is just for
comparison reasons, i.e. how would a complete random generation look like).
Bonus task (which will be very helpful for the next assignment):
• Use an LSTM as described here (but without the embeddings) to train the model.
• Feed the whole word at once to the LSTM (i.e. not a single characters at a time).
• Generate some words to see the results

Do not expect the models to generate always meaningful words. The model should
output words that are in some sense similar to valid words and which look much closer
to real words than randomly generated output
